# 1:Mesh for a brak disc
This part is from Jørgen S. Dokken blog: Mesh generation use gmsh API 
https://jsdokken.com/src/tutorial_gmsh.html



In [9]:
import gmsh
import sys
import math
import os
import numpy as np

gmsh.initialize()
gmsh.model.add("Disc 3D")

B, r_outer, r_inner = 20, 300, 200

# x (double), y (double), z (double), dx (double), dy (double), dz (double),
# r (double), tag = -1 (integer), angle = 2*pi (double)
outer_disc  = gmsh.model.occ.addCylinder(0,0,0,  0, 0, B,  r_outer)
inner_disc  = gmsh.model.occ.addCylinder(0,0,0,  0, 0, B,  r_inner)

#[(3, )]  3 means dimension, cut the common place
disk = gmsh.model.occ.cut([(3, outer_disc)], [(3, inner_disc)])

gmsh.model.occ.synchronize()

# Add physical group, if do not have this step, below 
# https://gitlab.onelab.info/gmsh/gmsh/blob/master/tutorials/python/t1.py#L115

# Volumes: 1
volumes = gmsh.model.occ.getEntities(dim=3)
gmsh.model.addPhysicalGroup(3, volumes[0], 1)

# Surfaces: 4 inner  5 outer  6 up  7 below 
surfaces = gmsh.model.occ.getEntities(dim=2)
gmsh.model.addPhysicalGroup(2, surfaces[0], 4)
gmsh.model.addPhysicalGroup(2, surfaces[1], 5)
gmsh.model.addPhysicalGroup(2, surfaces[2], 6)
gmsh.model.addPhysicalGroup(2, surfaces[3], 7)
   

# Mesh

gmsh.option.setNumber("Mesh.MeshSizeMin", 5)
gmsh.option.setNumber("Mesh.MeshSizeMax", 5)
#gmsh.model.occ.mesh.setSize = 0.001
gmsh.model.mesh.generate(3)
gmsh.write("disc.msh")

# Launch the GUI to see the results:
if '-nopopup' not in sys.argv:
    gmsh.fltk.run()
gmsh.finalize()


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 4 (Circle)
Info    : [ 20%] Meshing curve 5 (Line)
Info    : [ 40%] Meshing curve 6 (Circle)
Info    : [ 50%] Meshing curve 7 (Circle)
Info    : [ 70%] Meshing curve 8 (Line)
Info    : [ 90%] Meshing curve 9 (Circle)
Info    : Done meshing 1D (Wall 0.000931937s, CPU 0.00089s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 4 (Cylinder, Frontal-Delaunay)
Info    : [ 30%] Meshing surface 5 (Cylinder, Frontal-Delaunay)


Info    : [ 50%] Meshing surface 6 (Plane, Frontal-Delaunay)
Info    : [ 80%] Meshing surface 7 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.655392s, CPU 0.58599s)
Info    : Meshing 3D...
Info    : 3D Meshing 1 volume with 1 connected component
Info    : Tetrahedrizing 18056 nodes...
Info    : Done tetrahedrizing 18064 nodes (Wall 0.250192s, CPU 0.226196s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :  - Identifying boundary edges
Info    :  - Recovering boundary
Info    : Done reconstructing mesh (Wall 0.8718s, CPU 0.825883s)
Info    : Found volume 1
Info    : It. 0 - 0 nodes created - worst tet radius 2.21797 (nodes removed 0 0)
Info    : It. 500 - 500 nodes created - worst tet radius 2.08248 (nodes removed 0 0)
Info    : It. 1000 - 1000 nodes created - worst tet radius 1.92648 (nodes removed 0 0)
Info    : It. 1500 - 1500 nodes created - worst tet radius 1.6079 (nodes removed 0 0)
Info    : It. 2000 - 2000 nodes created - worst tet radi

# Here try to delete the line
dels = gmsh.model.occ.getEntities(dim=1)

gmsh.model.occ.remove(dels)  # Delete outside parts recursively



In [10]:
print('Surfaces is', surfaces)
print('Volumes is', volumes)

Surfaces is [(2, 4), (2, 5), (2, 6), (2, 7)]
Volumes is [(3, 1)]


# 2: Mesh plot

THis part is from FEniCSx tutorial: Plotting the mesh using pyvista

https://jsdokken.com/dolfinx-tutorial/chapter1/fundamentals_code.html

In [11]:
from dolfinx.io import gmshio
from mpi4py import MPI
import pyvista
from dolfinx import plot

# try a 3d mesh from gmsh, it works
domain, cell_markers, facet_markers = gmshio.read_from_msh("disc.msh", MPI.COMM_WORLD, 0, gdim=3)
gdim = 3
tdim = gdim -1 
pyvista.start_xvfb()
# below gdim change to tdim is plot in 2D
topology, cell_types, geometry = plot.vtk_mesh(domain, gdim)

grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()

plotter.show()

Info    : Reading 'disc.msh'...
Info    : 15 entities
Info    : 29049 nodes
Info    : 157492 elements
Info    : Done reading 'disc.msh'                                          


Widget(value='<iframe src="http://localhost:38145/index.html?ui=P_0x7ff6f49e2650_0&reconnect=auto" class="pyvi…